In [8]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import metrics
import requests, os
from zipfile import ZipFile
import shutil
from sklearn.cluster import MiniBatchKMeans 
import copy
from sklearn import svm as SVM

Consts

In [9]:
ZIP_PATH = "http://people.csail.mit.edu/torralba/code/spatialenvelope/spatial_envelope_256x256_static_8outdoorcategories.zip"

Helper Functions

In [10]:
'''
Download and unzip file
'''
def load_files():
    filename = ZIP_PATH.split("/")[-1]
    with open(filename, "wb") as f:
        r = requests.get(ZIP_PATH)
        f.write(r.content)
    foldername = filename.split(".")[0]
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()
        nameList = zip_ref.namelist()
        imgs = [name.split("/")[-1] for name in nameList if name.endswith('.jpg')]
        forest_images = [name for name in imgs if name.startswith('forest_')]
        mountain_images = [name for name in imgs if name.startswith('mountain_')]
    shutil.move(foldername, 'images')
    return forest_images,mountain_images


In [11]:
def prepare_set(image_names):
    num_imgs = len(image_names)
    split_index = int(num_imgs*0.8)
    train_set = image_names[:split_index-1]
    test_set = image_names[split_index:]
    return train_set, test_set

In [12]:
images_set1,images_set2 = load_files()
train_set_1, test_set_1 = prepare_set(images_set1)
train_set_2, test_set_2 = prepare_set(images_set2)
train_set = train_set_1+train_set_2
test_set = test_set_1+test_set_2

In [13]:
def get_image_by_name(filename):
    filename = 'images//' + filename
    image = cv2.imread(filename)
    return image


1. Compute dense SIFT from the images

In [83]:
def get_danse_SIFT(filename):
    image = get_image_by_name(filename)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    key_point = [cv2.KeyPoint(x, y, 8) for x in range(0, gray_image.shape[0], 8) for y in range(0, gray_image.shape[1], 8)]
    dense_shift = sift.compute(image, key_point)[1]
    return dense_shift.astype(float) 

In [101]:
#def get_dense_sift_set(image_set):
#    set_x = []
#    set_y = []
#    for filename in image_set:
#        res = get_danse_SIFT(filename)
#        set_x.append(res)
#        set_y.append(filename.split('_')[0])     
#    return set_x, set_y

In [102]:
#x_train, y_train = get_dense_sift_set(train_set)
#x_test, y_test = get_dense_sift_set(test_set)

In [103]:
#print(x_train)
#print(y_train)

In [104]:
def get_all_hists(x_train):
    # converts to 100 points vector using histogram of denseSift
    ss, w = 8, 128
    numOfLines = int((256 / ss) * (256 / ss))
    h = int(int(len(x_train)) * numOfLines)
    lines = np.zeros((h, w))
    cCounter = 0
    for c in x_train:
        denseSift = get_danse_SIFT(c)
        denseSiftLength = len(denseSift)
        for index in range(denseSiftLength):
            lines[int(numOfLines * cCounter + index)] = denseSift[index]
        cCounter += 1
    return lines

In [105]:
def calculate_kmeans(all_train_desc, k):
    kmeans = MiniBatchKMeans (n_clusters=k, random_state=0, batch_size=1000, max_iter=100000)
    kmeans.fit(all_train_desc)
    return kmeans

In [106]:
k = 100
all_hists_x = get_all_hists(train_set)
kmeans = calculate_kmeans(all_hists_x, k)


In [114]:
# form training set histograms for each training image using BoW representation
def calculate_histogram(x_train, kmeans, k):
    hist_set = []
    set_y = []
    for elm in x_train:
        predict = kmeans.predict(get_danse_SIFT(elm))
        hist, _ = np.histogram(predict, bins=np.arange(100), density=True)
        hist_set.append(np.asarray(hist))
        set_y.append(elm.split('_')[0])     
    return np.array(hist_set), np.array(set_y)

In [115]:
train_hist, train_y  = calculate_histogram(train_set, kmeans, k)
test_hist, test_y = calculate_histogram(test_set, kmeans, k)

In [125]:
def train_svm(x, y):
    # Train an SVM classifier
    svm = SVM.SVC(C=10.0, cache_size=4000)
    svm.fit(x, y)    
    return svm

In [126]:
print(train_hist.shape)
print(train_y.shape)
svm = train_svm(train_hist,train_y)


(559, 99)
(559,)


In [127]:
#for image in test_hist:
predict = svm.predict(test_hist)
#print(predict)
accuracyScore = metrics.accuracy_score(predict, test_y) * 100
print("the Prediction Accuracy Score is: " + str(accuracyScore))

the Prediction Accuracy Score is: 95.74468085106383


In [128]:
#os.rmdir(foldername)